<a href="https://colab.research.google.com/github/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/blob/main/ABC_XYZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from io import BytesIO

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Descargar el archivo
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer las hojas del Excel
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')

# Asegurar que la columna ITEM esté como índice o string limpio
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()

# Eliminar espacios innecesarios en nombres de columnas
forecast_df.columns = forecast_df.columns.str.strip()

# Sumar las ventas mensuales para obtener venta anual
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Convertir precios a número (eliminando '$' y comas)
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)

# Unir datos: Total de ventas con precio
merged_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')

# Calcular valor total en dólares (ventas * precio)
merged_df['Valor_Total_Dolares'] = merged_df['Total_Ventas_Anual'] * merged_df['Precio']

# Ordenar por valor total descendente
merged_df = merged_df.sort_values(by='Valor_Total_Dolares', ascending=False).reset_index(drop=True)

# Calcular el total general
total_valor = merged_df['Valor_Total_Dolares'].sum()

# Calcular porcentaje acumulado
merged_df['Porcentaje_Acumulado'] = merged_df['Valor_Total_Dolares'].cumsum() / total_valor * 100

# Clasificación ABC
def clasificar_abc(porcentaje_acumulado):
    if porcentaje_acumulado <= 60:
        return 'A'
    elif porcentaje_acumulado <= 80:
        return 'B'
    else:
        return 'C'

merged_df['Clase_ABC'] = merged_df['Porcentaje_Acumulado'].apply(clasificar_abc)

# Mostrar resultados
print("Clasificación ABC de Productos")
print("="*60)
print(merged_df[['ITEM', 'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares', 'Porcentaje_Acumulado', 'Clase_ABC']])

Clasificación ABC de Productos
         ITEM  Total_Ventas_Anual  Precio  Valor_Total_Dolares  \
0     ITEM 12             4895280    7.82          38281089.60   
1      ITEM 2             8292408    4.17          34579341.36   
2      ITEM 1             4827108    6.84          33017418.72   
3      ITEM 7             7351524    4.35          31979129.40   
4    ITEM 311             5234976    5.66          29629964.16   
..        ...                 ...     ...                  ...   
655  ITEM 472             2763960    1.62           4477615.20   
656  ITEM 625             2355924    1.87           4405577.88   
657  ITEM 624             2399628    1.72           4127360.16   
658  ITEM 655             2364588    1.56           3688757.28   
659  ITEM 639             3795540    0.80           3036432.00   

     Porcentaje_Acumulado Clase_ABC  
0                0.407538         A  
1                0.775668         A  
2                1.127169         A  
3                1.46761

In [2]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Descargar el archivo
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer la hoja 'Historico'
historico_df = pd.read_excel(excel_data, sheet_name='Historico')

# Asegurar que ITEM es string limpio
historico_df['ITEM'] = historico_df['ITEM'].astype(str).str.strip()

# Eliminar espacios en nombres de columnas
historico_df.columns = historico_df.columns.str.strip()

# Identificar columnas de meses (excluyendo 'ITEM')
month_cols = [col for col in historico_df.columns if col != 'ITEM']

# Convertir todos los valores numéricos a float
for col in month_cols:
    historico_df[col] = pd.to_numeric(historico_df[col], errors='coerce')

# Función para limpieza de datos por ítem
def limpiar_datos(item_data):
    # Reemplazar valores <= 0 por la mediana del ítem
    median_val = item_data.median()
    item_data_clean = item_data.clip(lower=median_val)  # Esto reemplaza valores < median_val por median_val
    return item_data_clean

# Aplicar limpieza por ítem
data_cleaned = historico_df.copy()
for col in month_cols:
    data_cleaned[col] = data_cleaned.groupby('ITEM')[col].transform(limpiar_datos)

# Aplicar Winzorización: reemplazar valores fuera de 5% y 95%
def winzorize_series(series):
    p5 = series.quantile(0.05)
    p95 = series.quantile(0.95)
    return series.clip(lower=p5, upper=p95)

# Aplicar Winzorización por ítem
for col in month_cols:
    data_cleaned[col] = data_cleaned.groupby('ITEM')[col].transform(winzorize_series)

# Calcular coeficiente de variación (CV) para cada ítem
cv_data = []
for idx, row in data_cleaned.iterrows():
    item = row['ITEM']
    values = row[month_cols].dropna().values
    if len(values) == 0:
        cv = np.nan
    else:
        mean_val = np.mean(values)
        std_val = np.std(values, ddof=1)  # desviación estándar muestral
        if mean_val == 0:
            cv = np.inf  # si media es cero, CV no definido
        else:
            cv = std_val / mean_val
    cv_data.append({'ITEM': item, 'CV': cv})

cv_df = pd.DataFrame(cv_data)

# Ordenar por CV ascendente
cv_df = cv_df.sort_values(by='CV', ascending=True).reset_index(drop=True)

# Calcular percentiles
cv_df['Percentil'] = cv_df['CV'].rank(pct=True) * 100

# Clasificación XYZ
def clasificar_xyz(percentil):
    if percentil <= 33:
        return 'X'
    elif percentil <= 67:
        return 'Y'
    else:
        return 'Z'

cv_df['Clase_XYZ'] = cv_df['Percentil'].apply(clasificar_xyz)

# Mostrar resultados
print("Clasificación XYZ por Coeficiente de Variación")
print("="*70)
print(cv_df[['ITEM', 'CV', 'Percentil', 'Clase_XYZ']].round(4))

TypeError: arg must be a list, tuple, 1-d array, or Series

In [3]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Descargar el archivo
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer la hoja 'Historico'
historico_df = pd.read_excel(excel_data, sheet_name='Historico')

# Asegurar que ITEM es string limpio
historico_df['ITEM'] = historico_df['ITEM'].astype(str).str.strip()

# Eliminar espacios en nombres de columnas
historico_df.columns = historico_df.columns.str.strip()

# Identificar columnas de meses (todas excepto 'ITEM')
month_cols = [col for col in historico_df.columns if col != 'ITEM']

# Convertir cada columna de meses a numérico, celda por celda
for col in month_cols:
    historico_df[col] = historico_df[col].astype(str).str.strip()  # Convertir a string y limpiar
    historico_df[col] = pd.to_numeric(historico_df[col], errors='coerce')  # Convertir a numérico, NaN si falla

# Función para limpiar datos por ítem: reemplazar ceros/negativos por mediana
def limpiar_datos(group):
    values = group[month_cols].values.flatten()
    median_val = np.median(values[~np.isnan(values)])  # Mediana sin NaN
    group[month_cols] = group[month_cols].clip(lower=median_val, axis=0)  # Reemplazar valores bajos
    return group

# Aplicar limpieza por ITEM
data_cleaned = historico_df.groupby('ITEM', group_keys=False).apply(limpiar_datos)

# Winzorización: reemplazar extremos por percentiles 5% y 95% por ITEM
def winzorize_group(group):
    data = group[month_cols].values.flatten()
    p5 = np.nanpercentile(data, 5)
    p95 = np.nanpercentile(data, 95)
    group[month_cols] = np.clip(group[month_cols], p5, p95)
    return group

data_cleaned = data_cleaned.groupby('ITEM', group_keys=False).apply(winzorize_group)

# Calcular coeficiente de variación (CV) por ITEM
cv_results = []
for item, group in data_cleaned.groupby('ITEM'):
    values = group[month_cols].values.flatten()
    values = values[~np.isnan(values)]  # Eliminar NaN
    if len(values) == 0:
        cv = np.nan
    else:
        mean_val = np.mean(values)
        std_val = np.std(values, ddof=1)
        cv = std_val / mean_val if mean_val != 0 else np.inf
    cv_results.append({'ITEM': item, 'CV': cv})

# Crear DataFrame de CV
cv_df = pd.DataFrame(cv_results)

# Ordenar por CV
cv_df = cv_df.sort_values(by='CV', ascending=True).reset_index(drop=True)

# Calcular percentil
cv_df['Percentil'] = (cv_df['CV'].rank(method='min') - 1) / len(cv_df) * 100

# Clasificación XYZ
def clasificar_xyz(percentil):
    if percentil <= 33:
        return 'X'
    elif percentil <= 67:
        return 'Y'
    else:
        return 'Z'

cv_df['Clase_XYZ'] = cv_df['Percentil'].apply(clasificar_xyz)

# Mostrar resultados
print("Clasificación XYZ por Coeficiente de Variación")
print("="*70)
print(cv_df[['ITEM', 'CV', 'Percentil', 'Clase_XYZ']].round(4))

AttributeError: 'DataFrame' object has no attribute 'str'

In [4]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Descargar el archivo
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer la hoja 'Historico'
historico_df = pd.read_excel(excel_data, sheet_name='Historico')

# Asegurar que ITEM es string limpio
historico_df['ITEM'] = historico_df['ITEM'].astype(str).str.strip()

# Limpiar nombres de columnas
historico_df.columns = historico_df.columns.str.strip()

# Columnas de meses
month_cols = [col for col in historico_df.columns if col != 'ITEM']

# Función para convertir un valor a número, limpiando espacios y caracteres extraños
def clean_value(x):
    if pd.isna(x):
        return np.nan
    try:
        # Convertir a string, limpiar espacios y caracteres extraños
        x_str = str(x).strip().replace('$', '').replace(',', '').replace(' ', '')
        return float(x_str)
    except ValueError:
        return np.nan  # Si no se puede convertir, NaN

# Aplicar limpieza a cada columna de meses
for col in month_cols:
    historico_df[col] = historico_df[col].apply(clean_value)

# Función para limpiar datos por grupo (reemplazar <=0 por mediana)
def limpiar_grupo(group):
    values = group[month_cols].values.flatten()
    valid_values = values[~np.isnan(values) & (values > 0)]  # Solo valores positivos y no NaN
    if len(valid_values) == 0:
        median_val = 1  # valor por defecto si todo es cero o negativo
    else:
        median_val = np.median(valid_values)

    # Reemplazar valores <= 0 o NaN por la mediana
    group[month_cols] = group[month_cols].applymap(lambda x: median_val if pd.isna(x) or x <= 0 else x)
    return group

# Aplicar limpieza por ITEM
data_cleaned = historico_df.groupby('ITEM', group_keys=False).apply(limpiar_grupo)

# Winzorización por ITEM: reemplazar valores extremos por percentiles 5% y 95%
def winzorizar_grupo(group):
    values = group[month_cols].values.flatten()
    valid_values = values[~np.isnan(values)]
    if len(valid_values) < 2:
        return group  # No hacer nada si no hay suficientes datos
    p5 = np.percentile(valid_values, 5)
    p95 = np.percentile(valid_values, 95)
    group[month_cols] = group[month_cols].applymap(lambda x: p5 if x < p5 else (p95 if x > p95 else x))
    return group

data_cleaned = data_cleaned.groupby('ITEM', group_keys=False).apply(winzorizar_grupo)

# Calcular coeficiente de variación (CV) por ITEM
cv_results = []
for item, group in data_cleaned.groupby('ITEM'):
    values = group[month_cols].values.flatten()
    values = values[~np.isnan(values)]  # Eliminar NaN
    if len(values) == 0 or np.mean(values) == 0:
        cv = np.nan
    else:
        cv = np.std(values, ddof=1) / np.mean(values)
    cv_results.append({'ITEM': item, 'CV': cv})

# Crear DataFrame
cv_df = pd.DataFrame(cv_results)
cv_df = cv_df.dropna().sort_values(by='CV', ascending=True).reset_index(drop=True)

# Calcular percentil
cv_df['Percentil'] = (cv_df['CV'].rank(method='min') - 1) / len(cv_df) * 100

# Clasificación XYZ
def clasificar_xyz(percentil):
    if percentil <= 33:
        return 'X'
    elif percentil <= 67:
        return 'Y'
    else:
        return 'Z'

cv_df['Clase_XYZ'] = cv_df['Percentil'].apply(clasificar_xyz)

# Mostrar resultados
print("Clasificación XYZ por Coeficiente de Variación")
print("="*70)
print(cv_df[['ITEM', 'CV', 'Percentil', 'Clase_XYZ']].round(4))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [5]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Descargar el archivo
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer la hoja 'Historico'
historico_df = pd.read_excel(excel_data, sheet_name='Historico', dtype=object)  # Leer todo como objeto

# Limpiar nombres de columnas y ITEM
historico_df.columns = historico_df.columns.str.strip()
historico_df['ITEM'] = historico_df['ITEM'].astype(str).str.strip()

# Columnas de meses
month_cols = [col for col in historico_df.columns if col != 'ITEM']

# Función para convertir un valor a número limpio
def clean_value(x):
    if x is None or pd.isna(x):
        return np.nan
    try:
        # Convertir a string, limpiar
        x_str = str(x).strip().replace('$', '').replace(',', '').replace(' ', '')
        if x_str == '' or x_str.lower() in ['na', 'nan', '-']:
            return np.nan
        return float(x_str)
    except (ValueError, TypeError):
        return np.nan

# Reconstruir el DataFrame con valores limpios
for col in month_cols:
    historico_df[col] = historico_df[col].apply(lambda x: clean_value(x))

# Asegurarnos de que todo sea numérico
for col in month_cols:
    historico_df[col] = pd.to_numeric(historico_df[col], errors='coerce')

# Función para limpiar cada grupo (ITEM): reemplazar <=0 o NaN por mediana positiva
def limpiar_grupo(group):
    values = group[month_cols].values.flatten()
    valid_values = values[(~np.isnan(values)) & (values > 0)]
    if len(valid_values) == 0:
        median_val = 1.0
    else:
        median_val = np.median(valid_values)
    # Reemplazar valores inválidos
    group[month_cols] = group[month_cols].applymap(
        lambda x: median_val if pd.isna(x) or x <= 0 else x
    )
    return group

# Aplicar limpieza por ITEM
data_cleaned = historico_df.groupby('ITEM', group_keys=False).apply(limpiar_grupo)

# Winzorización por percentiles 5% y 95% por ITEM
def winzorizar_grupo(group):
    values = group[month_cols].values.flatten()
    valid_values = values[~np.isnan(values)]
    if len(valid_values) < 2:
        return group
    p5 = np.percentile(valid_values, 5)
    p95 = np.percentile(valid_values, 95)
    group[month_cols] = group[month_cols].applymap(
        lambda x: p5 if x < p5 else (p95 if x > p95 else x)
    )
    return group

data_cleaned = data_cleaned.groupby('ITEM', group_keys=False).apply(winzorizar_grupo)

# Calcular CV por ITEM
cv_results = []
for item, group in data_cleaned.groupby('ITEM'):
    values = group[month_cols].values.flatten()
    values = values[~np.isnan(values)]
    if len(values) == 0:
        continue
    mean_val = np.mean(values)
    std_val = np.std(values, ddof=1)
    cv = std_val / mean_val if mean_val != 0 else np.inf
    cv_results.append({'ITEM': item, 'CV': cv})

# Crear DataFrame final
cv_df = pd.DataFrame(cv_results)
cv_df = cv_df.replace([np.inf, -np.inf], np.nan).dropna().sort_values(by='CV', ascending=True).reset_index(drop=True)

# Calcular percentil
cv_df['Percentil'] = (cv_df['CV'].rank(method='min') - 1) / len(cv_df) * 100

# Clasificación XYZ
def clasificar_xyz(percentil):
    if percentil <= 33:
        return 'X'
    elif percentil <= 67:
        return 'Y'
    else:
        return 'Z'

cv_df['Clase_XYZ'] = cv_df['Percentil'].apply(clasificar_xyz)

# Mostrar resultados
print("Clasificación XYZ - Demanda por estabilidad")
print("=" * 60)
print(cv_df[['ITEM', 'CV', 'Percentil', 'Clase_XYZ']].round(4))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [6]:
import pandas as pd
import numpy as np

# Supongamos que ya tienes el DataFrame `historico_df` cargado desde tu archivo Excel.
# Si no, asegúrate de cargarlo correctamente antes de ejecutar este código.

# Limpiar nombres de columnas y ITEM
historico_df.columns = historico_df.columns.str.strip()
historico_df['ITEM'] = historico_df['ITEM'].astype(str).str.strip()

# Columnas de meses
month_cols = [col for col in historico_df.columns if col != 'ITEM']

# Función para convertir un valor a número limpio
def clean_value(x):
    if x is None or pd.isna(x):
        return np.nan
    try:
        x_str = str(x).strip().replace('$', '').replace(',', '').replace(' ', '')
        if x_str == '' or x_str.lower() in ['na', 'nan', '-']:
            return np.nan
        return float(x_str)
    except (ValueError, TypeError):
        return np.nan

# Reconstruir el DataFrame con valores limpios
for col in month_cols:
    historico_df[col] = historico_df[col].apply(lambda x: clean_value(x))

# Asegurarnos de que todo sea numérico
for col in month_cols:
    historico_df[col] = pd.to_numeric(historico_df[col], errors='coerce')

# Función para limpiar cada grupo (ITEM): reemplazar <=0 o NaN por mediana positiva
def limpiar_grupo(group):
    values = group[month_cols].values.flatten()
    valid_values = values[(~np.isnan(values)) & (values > 0)]
    if len(valid_values) == 0:
        median_val = 1.0
    else:
        median_val = np.median(valid_values)
    group[month_cols] = group[month_cols].applymap(lambda x: median_val if pd.isna(x) or x <= 0 else x)
    return group

# Aplicar limpieza por ITEM
data_cleaned = historico_df.groupby('ITEM', group_keys=False).apply(limpiar_grupo)

# Winzorización por percentiles 5% y 95% por ITEM
def winzorizar_grupo(group):
    values = group[month_cols].values.flatten()
    valid_values = values[~np.isnan(values)]
    if len(valid_values) < 2:
        return group
    p5 = np.percentile(valid_values, 5)
    p95 = np.percentile(valid_values, 95)
    group[month_cols] = group[month_cols].applymap(lambda x: p5 if x < p5 else (p95 if x > p95 else x))
    return group

data_cleaned = data_cleaned.groupby('ITEM', group_keys=False).apply(winzorizar_grupo)

# Calcular CV por ITEM
cv_results = []
for item, group in data_cleaned.groupby('ITEM'):
    values = group[month_cols].values.flatten()
    values = values[~np.isnan(values)]
    if len(values) == 0:
        continue
    mean_val = np.mean(values)
    std_val = np.std(values, ddof=1)
    cv = std_val / mean_val if mean_val != 0 else np.inf
    cv_results.append({'ITEM': item, 'CV': cv})

# Crear DataFrame final
cv_df = pd.DataFrame(cv_results)
cv_df = cv_df.replace([np.inf, -np.inf], np.nan).dropna().sort_values(by='CV', ascending=True).reset_index(drop=True)

# Calcular percentil
cv_df['Percentil'] = (cv_df['CV'].rank(method='min') - 1) / len(cv_df) * 100

# Clasificación XYZ
def clasificar_xyz(percentil):
    if percentil <= 33:
        return 'X'
    elif percentil <= 67:
        return 'Y'
    else:
        return 'Z'

cv_df['Clase_XYZ'] = cv_df['Percentil'].apply(clasificar_xyz)

# Mostrar resultados
print("Clasificación XYZ - Demanda por estabilidad")
print("=" * 60)
print(cv_df[['ITEM', 'CV', 'Percentil', 'Clase_XYZ']].round(4))


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [8]:
import pandas as pd
import numpy as np

# Cargar el archivo Excel
historico_df = pd.read_excel('https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx', sheet_name='Historico', dtype=object)

# Limpiar nombres de columnas y ITEM
historico_df.columns = historico_df.columns.str.strip()
historico_df['ITEM'] = historico_df['ITEM'].astype(str).str.strip()

# Columnas de meses
month_cols = [col for col in historico_df.columns if col != 'ITEM']

# Función para convertir un valor a número limpio
def clean_value(x):
    if x is None or pd.isna(x):
        return np.nan
    try:
        x_str = str(x).strip().replace('$', '').replace(',', '').replace(' ', '')
        if x_str == '' or x_str.lower() in ['na', 'nan', '-']:
            return np.nan
        return float(x_str)
    except (ValueError, TypeError):
        return np.nan

# Reconstruir el DataFrame con valores limpios
for col in month_cols:
    historico_df[col] = historico_df[col].apply(lambda x: clean_value(x))

# Asegurarnos de que todo sea numérico
for col in month_cols:
    historico_df[col] = pd.to_numeric(historico_df[col], errors='coerce')

# Función para limpiar cada grupo (ITEM): reemplazar <=0 o NaN por mediana positiva
def limpiar_grupo(group):
    values = group[month_cols].values.flatten()
    valid_values = values[(~np.isnan(values)) & (values > 0)]
    if len(valid_values) == 0:
        median_val = 1.0
    else:
        median_val = np.median(valid_values)
    group[month_cols] = group[month_cols].applymap(lambda x: median_val if pd.isna(x) or x <= 0 else x)
    return group

# Aplicar limpieza por ITEM
data_cleaned = historico_df.groupby('ITEM', group_keys=False).apply(limpiar_grupo)

# Winzorización por percentiles 5% y 95% por ITEM
def winzorizar_grupo(group):
    values = group[month_cols].values.flatten()
    valid_values = values[~np.isnan(values)]
    if len(valid_values) < 2:
        return group
    p5 = np.percentile(valid_values, 5)
    p95 = np.percentile(valid_values, 95)
    group[month_cols] = group[month_cols].applymap(lambda x: p5 if x < p5 else (p95 if x > p95 else x))
    return group

data_cleaned = data_cleaned.groupby('ITEM', group_keys=False).apply(winzorizar_grupo)

# Calcular CV por ITEM
cv_results = []
for item, group in data_cleaned.groupby('ITEM'):
    values = group[month_cols].values.flatten()
    values = values[~np.isnan(values)]
    if len(values) == 0:
        continue
    mean_val = np.mean(values)
    std_val = np.std(values, ddof=1)
    cv = std_val / mean_val if mean_val != 0 else np.inf
    cv_results.append({'ITEM': item, 'CV': cv})

# Crear DataFrame final
cv_df = pd.DataFrame(cv_results)
cv_df = cv_df.replace([np.inf, -np.inf], np.nan).dropna().sort_values(by='CV', ascending=True).reset_index(drop=True)

# Calcular percentil
cv_df['Percentil'] = (cv_df['CV'].rank(method='min') - 1) / len(cv_df) * 100

# Clasificación XYZ
def clasificar_xyz(percentil):
    if percentil <= 33:
        return 'X'
    elif percentil <= 67:
        return 'Y'
    else:
        return 'Z'

cv_df['Clase_XYZ'] = cv_df['Percentil'].apply(clasificar_xyz)

# Mostrar resultados
print("Clasificación XYZ - Demanda por estabilidad")
print("=" * 60)
print(cv_df[['ITEM', 'CV', 'Percentil', 'Clase_XYZ']].round(4))


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [9]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

# Descargar datos
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"
df = pd.read_excel(url, sheet_name="Historico", index_col="ITEM")

# Seleccionar últimos 18 meses (feb-2024 a jul-2025)
cols = df.columns[df.columns.get_loc("feb-24"):df.columns.get_loc("jul-25")+1]
data = df[cols].copy()

# Función para limpiar y winsorizar por fila
def clean_row(row):
    # Copiar fila
    clean = row.copy()

    # Reemplazar valores <=0 con la mediana
    median = clean[clean > 0].median()
    median = median if not np.isnan(median) else 0
    clean[clean <= 0] = median

    # Winsorización (solo si hay variabilidad)
    if clean.nunique() > 1:
        clean[:] = winsorize(clean, limits=[0.05, 0.05], nan_policy='omit')
    return clean

# Aplicar limpieza
data_clean = data.apply(clean_row, axis=1)

# Calcular coeficiente de variación (CV)
mean = data_clean.mean(axis=1)
std = data_clean.std(axis=1)
cv = (std / mean).replace(np.inf, 1000).fillna(1000)  # Manejar casos con media=0

# Clasificación XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)
clasificacion = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])

# Resultado final
resultado = pd.DataFrame({
    'Media': mean.round(2),
    'Desviación': std.round(2),
    'CV': cv.round(2),
    'Clasificación': clasificacion
})

print("Límites de clasificación:")
print(f"- X: CV ≤ {p33:.2f} (Percentil 33)")
print(f"- Y: {p33:.2f} < CV ≤ {p67:.2f} (Percentil 67)")
print(f"- Z: CV > {p67:.2f}\n")
print("Primeras 10 clasificaciones:")
print(resultado.head(10))

KeyError: 'feb-24'

In [10]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

# Descargar datos
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"
df = pd.read_excel(url, sheet_name="Historico", index_col="ITEM")

# Seleccionar últimos 18 meses por posición (feb-2024 a jul-2025)
data = df.iloc[:, -18:].copy()

# Función para limpiar y winsorizar por fila
def clean_row(row):
    # Copiar fila
    clean = row.copy().astype(float)

    # Reemplazar valores <=0 con la mediana
    positive_vals = clean[clean > 0]
    median_val = positive_vals.median() if not positive_vals.empty else 0
    clean[clean <= 0] = median_val

    # Winsorización solo si hay suficientes valores distintos
    if clean.nunique() > 1:
        try:
            clean[:] = winsorize(clean, limits=[0.05, 0.05])
        except ValueError:
            pass  # Si falla, mantener valores originales

    return clean

# Aplicar limpieza
data_clean = data.apply(clean_row, axis=1)

# Calcular coeficiente de variación (CV)
mean = data_clean.mean(axis=1)
std = data_clean.std(axis=1)
cv = std / mean

# Manejar casos especiales
cv = cv.replace([np.inf, -np.inf], 1000)  # Para medias = 0
cv = cv.fillna(1000)  # Para filas sin datos

# Clasificación XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)
clasificacion = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])

# Resultado final
resultado = pd.DataFrame({
    'Media': mean.round(2),
    'Desviación': std.round(2),
    'CV': cv.round(4),
    'Clasificación': clasificacion
})

# Mostrar resultados
print("="*60)
print("Límites de clasificación:")
print(f"- X: CV ≤ {p33:.4f} (Percentil 33)")
print(f"- Y: {p33:.4f} < CV ≤ {p67:.4f} (Percentil 67)")
print(f"- Z: CV > {p67:.4f}")
print("="*60)
print(f"Total ítems clasificados: {len(resultado)}")
print(resultado['Clasificación'].value_counts())
print("="*60)
print("Ejemplo de ítems clasificados:")
print(resultado.sample(10, random_state=1))

Límites de clasificación:
- X: CV ≤ 0.1510 (Percentil 33)
- Y: 0.1510 < CV ≤ 0.1583 (Percentil 67)
- Z: CV > 0.1583
Total ítems clasificados: 660
Clasificación
Y    224
X    218
Z    218
Name: count, dtype: int64
Ejemplo de ítems clasificados:
             Media  Desviación      CV Clasificación
ITEM                                                
ITEM 548  29748.39     4272.86  0.1436             X
ITEM 354  17354.72     2620.09  0.1510             X
ITEM 500  23371.39     3534.81  0.1512             Y
ITEM 174  19585.56     3101.69  0.1584             Z
ITEM 242  27983.83     4185.15  0.1496             X
ITEM 342  30780.00     4638.80  0.1507             X
ITEM 648  21299.50     3503.52  0.1645             Z
ITEM 219  31940.06     4907.00  0.1536             Y
ITEM 121  44915.33     6956.74  0.1549             Y
ITEM 135  29912.11     4983.50  0.1666             Z


In [11]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats.mstats import winsorize

# 1. Descargar y cargar datos
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer todas las hojas necesarias
historico_df = pd.read_excel(excel_data, sheet_name='Historico', index_col='ITEM')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')

# 2. Clasificación ABC
# Limpiar y preparar datos para ABC
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Calcular ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar y convertir precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)

# Unir datos y calcular valor total
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y calcular acumulado
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
def clasificar_abc(porcentaje):
    if porcentaje <= 60: return 'A'
    elif porcentaje <= 80: return 'B'
    else: return 'C'

abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(clasificar_abc)

# 3. Clasificación XYZ
# Seleccionar últimos 18 meses
xyz_data = historico_df.iloc[:, -18:].copy()

# Función para limpiar datos
def clean_row(row):
    clean = row.copy().astype(float)
    positive_vals = clean[clean > 0]
    median_val = positive_vals.median() if not positive_vals.empty else 0
    clean[clean <= 0] = median_val

    if clean.nunique() > 1:
        try:
            clean[:] = winsorize(clean, limits=[0.05, 0.05])
        except:
            pass
    return clean

# Aplicar limpieza
xyz_clean = xyz_data.apply(clean_row, axis=1)

# Calcular coeficiente de variación
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Clasificar XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)
xyz_df = pd.DataFrame({
    'Media_Demanda': mean,
    'Desviacion_Demanda': std,
    'CV': cv
})
xyz_df['Clase_XYZ'] = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])

# 4. Combinar ABC y XYZ
# Preparar DataFrames para merge
abc_df = abc_df.set_index('ITEM')
xyz_df = xyz_df[['CV', 'Clase_XYZ']]  # Solo necesitamos estas columnas

# Hacer merge
final_df = abc_df.merge(xyz_df, left_index=True, right_index=True, how='inner')
final_df['Clasificacion_ABC_XYZ'] = final_df['Clase_ABC'] + final_df['Clase_XYZ']

# 5. Resultados
print("="*80)
print(f"Total productos clasificados: {len(final_df)}")
print("Distribución ABC:")
print(final_df['Clase_ABC'].value_counts())
print("\nDistribución XYZ:")
print(final_df['Clase_XYZ'].value_counts())
print("\nDistribución ABC-XYZ:")
print(final_df['Clasificacion_ABC_XYZ'].value_counts())
print("="*80)

# Mostrar ejemplo de resultados
print("Ejemplo de clasificación combinada:")
print(final_df.sample(10, random_state=1)[[
    'Total_Ventas_Anual', 'Valor_Total_Dolares', 'Porcentaje_Acumulado',
    'CV', 'Clase_ABC', 'Clase_XYZ', 'Clasificacion_ABC_XYZ'
]].round(2))

# 6. Exportar resultados (opcional)
final_df.to_excel("Clasificacion_ABC_XYZ.xlsx")
print("\nResultados exportados a 'Clasificacion_ABC_XYZ.xlsx'")
print("="*80)

TypeError: Object with dtype category cannot perform the numpy op add

In [12]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats.mstats import winsorize

# 1. Descargar y cargar datos
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer todas las hojas necesarias
historico_df = pd.read_excel(excel_data, sheet_name='Historico', index_col='ITEM')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')

# 2. Clasificación ABC
# Limpiar y preparar datos para ABC
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Calcular ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar y convertir precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)

# Unir datos y calcular valor total
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y calcular acumulado
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
def clasificar_abc(porcentaje):
    if porcentaje <= 60: return 'A'
    elif porcentaje <= 80: return 'B'
    else: return 'C'

abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(clasificar_abc)

# 3. Clasificación XYZ
# Seleccionar últimos 18 meses
xyz_data = historico_df.iloc[:, -18:].copy()

# Función para limpiar datos
def clean_row(row):
    clean = row.copy().astype(float)
    positive_vals = clean[clean > 0]
    median_val = positive_vals.median() if not positive_vals.empty else 0
    clean[clean <= 0] = median_val

    if clean.nunique() > 1:
        try:
            clean[:] = winsorize(clean, limits=[0.05, 0.05])
        except:
            pass
    return clean

# Aplicar limpieza
xyz_clean = xyz_data.apply(clean_row, axis=1)

# Calcular coeficiente de variación
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Clasificar XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)
xyz_df = pd.DataFrame({
    'Media_Demanda': mean,
    'Desviacion_Demanda': std,
    'CV': cv
})
xyz_df['Clase_XYZ'] = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])

# 4. Combinar ABC y XYZ
# Preparar DataFrames para merge
abc_df = abc_df.set_index('ITEM')
xyz_df = xyz_df[['CV', 'Clase_XYZ']]  # Solo necesitamos estas columnas

# Hacer merge
final_df = abc_df.merge(xyz_df, left_index=True, right_index=True, how='inner')

# CORRECCIÓN: Convertir categorías a strings antes de concatenar
final_df['Clase_ABC'] = final_df['Clase_ABC'].astype(str)
final_df['Clase_XYZ'] = final_df['Clase_XYZ'].astype(str)

# Crear clasificación combinada
final_df['Clasificacion_ABC_XYZ'] = final_df['Clase_ABC'] + final_df['Clase_XYZ']

# 5. Resultados
print("="*80)
print(f"Total productos clasificados: {len(final_df)}")
print("Distribución ABC:")
print(final_df['Clase_ABC'].value_counts())
print("\nDistribución XYZ:")
print(final_df['Clase_XYZ'].value_counts())
print("\nDistribución ABC-XYZ:")
print(final_df['Clasificacion_ABC_XYZ'].value_counts())
print("="*80)

# Mostrar ejemplo de resultados
print("Ejemplo de clasificación combinada:")
print(final_df.sample(10, random_state=1)[[
    'Total_Ventas_Anual', 'Valor_Total_Dolares', 'Porcentaje_Acumulado',
    'CV', 'Clase_ABC', 'Clase_XYZ', 'Clasificacion_ABC_XYZ'
]].round(2))

# 6. Exportar resultados (opcional)
final_df.to_excel("Clasificacion_ABC_XYZ.xlsx")
print("\nResultados exportados a 'Clasificacion_ABC_XYZ.xlsx'")
print("="*80)

Total productos clasificados: 660
Distribución ABC:
Clase_ABC
A    299
C    214
B    147
Name: count, dtype: int64

Distribución XYZ:
Clase_XYZ
Y    224
X    218
Z    218
Name: count, dtype: int64

Distribución ABC-XYZ:
Clasificacion_ABC_XYZ
AY    116
AZ    106
CX     87
AX     77
CY     68
CZ     59
BX     54
BZ     53
BY     40
Name: count, dtype: int64
Ejemplo de clasificación combinada:
          Total_Ventas_Anual  Valor_Total_Dolares  Porcentaje_Acumulado    CV  \
ITEM                                                                            
ITEM 114             2912004           9260172.72                 91.26  0.16   
ITEM 244             3959916          13028123.64                 67.90  0.16   
ITEM 570             3976404          10298886.36                 86.26  0.15   
ITEM 341             4993920          17129145.60                 39.07  0.16   
ITEM 427             4098360          15573768.00                 50.93  0.15   
ITEM 78              4286556          1

In [13]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats.mstats import winsorize

# 1. Descargar y cargar datos
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer todas las hojas necesarias
historico_df = pd.read_excel(excel_data, sheet_name='Historico', index_col='ITEM')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')

# 2. Clasificación ABC
# Limpiar y preparar datos para ABC
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Calcular ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar y convertir precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)

# Unir datos y calcular valor total
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y calcular acumulado
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
def clasificar_abc(porcentaje):
    if porcentaje <= 60: return 'A'
    elif porcentaje <= 80: return 'B'
    else: return 'C'

abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(clasificar_abc)

# 3. Clasificación XYZ
# Seleccionar últimos 18 meses
xyz_data = historico_df.iloc[:, -18:].copy()

# Función para limpiar datos
def clean_row(row):
    clean = row.copy().astype(float)
    positive_vals = clean[clean > 0]
    median_val = positive_vals.median() if not positive_vals.empty else 0
    clean[clean <= 0] = median_val

    if clean.nunique() > 1:
        try:
            clean[:] = winsorize(clean, limits=[0.05, 0.05])
        except:
            pass
    return clean

# Aplicar limpieza
xyz_clean = xyz_data.apply(clean_row, axis=1)

# Calcular coeficiente de variación
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Clasificar XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)
xyz_df = pd.DataFrame({
    'Media_Demanda': mean,
    'Desviacion_Demanda': std,
    'CV': cv
})
xyz_df['Clase_XYZ'] = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])

# 4. Combinar ABC y XYZ
# Preparar DataFrames para merge
abc_df = abc_df.set_index('ITEM')
xyz_df = xyz_df[['CV', 'Clase_XYZ']]  # Solo necesitamos estas columnas

# Hacer merge
final_df = abc_df.merge(xyz_df, left_index=True, right_index=True, how='inner')

# CORRECCIÓN: Convertir categorías a strings antes de concatenar
final_df['Clase_ABC'] = final_df['Clase_ABC'].astype(str)
final_df['Clase_XYZ'] = final_df['Clase_XYZ'].astype(str)

# Crear clasificación combinada
final_df['Clasificacion_ABC_XYZ'] = final_df['Clase_ABC'] + final_df['Clase_XYZ']

# 5. Resultados
print("="*80)
print(f"Total productos clasificados: {len(final_df)}")
print("Distribución ABC:")
print(final_df['Clase_ABC'].value_counts())
print("\nDistribución XYZ:")
print(final_df['Clase_XYZ'].value_counts())
print("\nDistribución ABC-XYZ:")
print(final_df['Clasificacion_ABC_XYZ'].value_counts())
print("="*80)

# Mostrar ejemplo de resultados
print("Ejemplo de clasificación combinada:")
print(final_df.sample(10, random_state=1)[[
    'Total_Ventas_Anual', 'Valor_Total_Dolares', 'Porcentaje_Acumulado',
    'CV', 'Clase_ABC', 'Clase_XYZ', 'Clasificacion_ABC_XYZ'
]].round(2))

# 6. Exportar resultados (opcional)
final_df.to_excel("Clasificacion_ABC_XYZ.xlsx")
print("\nResultados exportados a 'Clasificacion_ABC_XYZ.xlsx'")
print("="*80)

Total productos clasificados: 660
Distribución ABC:
Clase_ABC
A    299
C    214
B    147
Name: count, dtype: int64

Distribución XYZ:
Clase_XYZ
Y    224
X    218
Z    218
Name: count, dtype: int64

Distribución ABC-XYZ:
Clasificacion_ABC_XYZ
AY    116
AZ    106
CX     87
AX     77
CY     68
CZ     59
BX     54
BZ     53
BY     40
Name: count, dtype: int64
Ejemplo de clasificación combinada:
          Total_Ventas_Anual  Valor_Total_Dolares  Porcentaje_Acumulado    CV  \
ITEM                                                                            
ITEM 114             2912004           9260172.72                 91.26  0.16   
ITEM 244             3959916          13028123.64                 67.90  0.16   
ITEM 570             3976404          10298886.36                 86.26  0.15   
ITEM 341             4993920          17129145.60                 39.07  0.16   
ITEM 427             4098360          15573768.00                 50.93  0.15   
ITEM 78              4286556          1

In [14]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats.mstats import winsorize

# 1. Descargar y cargar datos
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer todas las hojas necesarias
historico_df = pd.read_excel(excel_data, sheet_name='Historico', index_col='ITEM')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')

# 2. Clasificación ABC
# Limpiar y preparar datos para ABC
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Calcular ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar y convertir precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)

# Unir datos y calcular valor total
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y calcular acumulado
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
def clasificar_abc(porcentaje):
    if porcentaje <= 60: return 'A'
    elif porcentaje <= 80: return 'B'
    else: return 'C'

abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(clasificar_abc)

# 3. Clasificación XYZ
# Seleccionar últimos 18 meses
xyz_data = historico_df.iloc[:, -18:].copy()

# Función para limpiar datos
def clean_row(row):
    clean = row.copy().astype(float)
    positive_vals = clean[clean > 0]
    median_val = positive_vals.median() if not positive_vals.empty else 0
    clean[clean <= 0] = median_val

    if clean.nunique() > 1:
        try:
            clean[:] = winsorize(clean, limits=[0.05, 0.05])
        except:
            pass
    return clean

# Aplicar limpieza
xyz_clean = xyz_data.apply(clean_row, axis=1)

# Calcular coeficiente de variación
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Clasificar XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)
xyz_df = pd.DataFrame({
    'ITEM': xyz_clean.index,
    'CV': cv
})
xyz_df['Clase_XYZ'] = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])

# 4. Combinar ABC y XYZ
# Seleccionar solo las columnas necesarias de ABC
abc_minimal = abc_df[['ITEM', 'Clase_ABC']]

# Hacer merge usando la columna ITEM
final_df = abc_minimal.merge(xyz_df[['ITEM', 'Clase_XYZ']], on='ITEM', how='inner')

# Crear clasificación combinada
final_df['Clasificacion_ABC_XYZ'] = final_df['Clase_ABC'] + final_df['Clase_XYZ']

# 5. Exportar resultado final (solo clasificaciones)
final_df.to_excel("Clasificacion_ABC_XYZ.xlsx", index=False)

print("="*80)
print("DataFrame final exportado con las columnas:")
print(final_df.columns.tolist())
print(f"\nTotal de ítems clasificados: {len(final_df)}")
print("\nEjemplo de clasificaciones:")
print(final_df.sample(5, random_state=1))
print("\nResultados exportados a 'Clasificacion_ABC_XYZ.xlsx'")
print("="*80)

ValueError: 'ITEM' is both an index level and a column label, which is ambiguous.

In [15]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats.mstats import winsorize

# 1. Descargar y cargar datos
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer todas las hojas necesarias
historico_df = pd.read_excel(excel_data, sheet_name='Historico')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')

# 2. Clasificación ABC
# Limpiar y preparar datos para ABC
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Calcular ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar y convertir precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)

# Unir datos y calcular valor total
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y calcular acumulado
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
def clasificar_abc(porcentaje):
    if porcentaje <= 60: return 'A'
    elif porcentaje <= 80: return 'B'
    else: return 'C'

abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(clasificar_abc)

# 3. Clasificación XYZ
# Seleccionar últimos 18 meses de datos históricos
xyz_data = historico_df.set_index('ITEM').iloc[:, -18:].copy()

# Función para limpiar datos
def clean_row(row):
    clean = row.copy().astype(float)
    positive_vals = clean[clean > 0]
    median_val = positive_vals.median() if not positive_vals.empty else 0
    clean[clean <= 0] = median_val

    if clean.nunique() > 1:
        try:
            clean[:] = winsorize(clean, limits=[0.05, 0.05])
        except:
            pass
    return clean

# Aplicar limpieza
xyz_clean = xyz_data.apply(clean_row, axis=1)

# Calcular coeficiente de variación
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Clasificar XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)
xyz_df = pd.DataFrame({
    'ITEM': xyz_clean.index,
    'CV': cv
})
xyz_df['Clase_XYZ'] = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])

# 4. Combinar ABC y XYZ
# Seleccionar solo las columnas necesarias de ABC
abc_minimal = abc_df[['ITEM', 'Clase_ABC']]

# Hacer merge usando la columna ITEM
final_df = abc_minimal.merge(xyz_df[['ITEM', 'Clase_XYZ']], on='ITEM', how='inner')

# Crear clasificación combinada
final_df['Clasificacion_ABC_XYZ'] = final_df['Clase_ABC'] + final_df['Clase_XYZ']

# 5. Exportar resultado final (solo clasificaciones)
final_df.to_excel("Clasificacion_ABC_XYZ.xlsx", index=False)

print("="*80)
print("DataFrame final exportado con las columnas:")
print(final_df.columns.tolist())
print(f"\nTotal de ítems clasificados: {len(final_df)}")
print("\nEjemplo de clasificaciones:")
print(final_df.sample(5, random_state=1))
print("\nResultados exportados a 'Clasificacion_ABC_XYZ.xlsx'")
print("="*80)

ValueError: 'ITEM' is both an index level and a column label, which is ambiguous.

In [16]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats.mstats import winsorize

# 1. Descargar y cargar datos
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer todas las hojas necesarias
historico_df = pd.read_excel(excel_data, sheet_name='Historico')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')

# 2. Clasificación ABC
# Limpiar y preparar datos para ABC
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Calcular ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar y convertir precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)

# Unir datos y calcular valor total
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y calcular acumulado
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
def clasificar_abc(porcentaje):
    if porcentaje <= 60: return 'A'
    elif porcentaje <= 80: return 'B'
    else: return 'C'

abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(clasificar_abc)

# 3. Clasificación XYZ
# Seleccionar últimos 18 meses de datos históricos
xyz_data = historico_df.set_index('ITEM').iloc[:, -18:].copy()

# Función para limpiar datos
def clean_row(row):
    clean = row.copy().astype(float)
    positive_vals = clean[clean > 0]
    median_val = positive_vals.median() if not positive_vals.empty else 0
    clean[clean <= 0] = median_val

    if clean.nunique() > 1:
        try:
            clean[:] = winsorize(clean, limits=[0.05, 0.05])
        except:
            pass
    return clean

# Aplicar limpieza
xyz_clean = xyz_data.apply(clean_row, axis=1)

# Calcular coeficiente de variación
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Clasificar XYZ - CORRECCIÓN AQUÍ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)

# Resetear el índice para convertir el índice 'ITEM' en una columna normal
xyz_df = pd.DataFrame({
    'ITEM': xyz_clean.index.tolist(),  # Convertir explícitamente a lista
    'CV': cv.values  # Usar .values para obtener los valores sin el índice
}).reset_index(drop=True)  # Resetear índice para evitar conflictos

# Aplicar clasificación XYZ
xyz_df['Clase_XYZ'] = pd.cut(cv.values, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])

# Convertir la columna ITEM a string para asegurar consistencia
xyz_df['ITEM'] = xyz_df['ITEM'].astype(str).str.strip()

# 4. Combinar ABC y XYZ
# Seleccionar solo las columnas necesarias de ABC
abc_minimal = abc_df[['ITEM', 'Clase_ABC']].copy()

# Asegurar que ambas columnas ITEM sean del mismo tipo
abc_minimal['ITEM'] = abc_minimal['ITEM'].astype(str).str.strip()

# Hacer merge usando la columna ITEM
final_df = abc_minimal.merge(xyz_df[['ITEM', 'Clase_XYZ']], on='ITEM', how='inner')

# Crear clasificación combinada
final_df['Clasificacion_ABC_XYZ'] = final_df['Clase_ABC'].astype(str) + final_df['Clase_XYZ'].astype(str)

# 5. Exportar resultado final (solo clasificaciones)
final_df.to_excel("Clasificacion_ABC_XYZ.xlsx", index=False)

print("="*80)
print("DataFrame final exportado con las columnas:")
print(final_df.columns.tolist())
print(f"\nTotal de ítems clasificados: {len(final_df)}")
print("\nEjemplo de clasificaciones:")
print(final_df.sample(min(5, len(final_df)), random_state=1))
print("\nDistribución de clasificaciones ABC-XYZ:")
print(final_df['Clasificacion_ABC_XYZ'].value_counts().sort_index())
print("\nResultados exportados a 'Clasificacion_ABC_XYZ.xlsx'")
print("="*80)

DataFrame final exportado con las columnas:
['ITEM', 'Clase_ABC', 'Clase_XYZ', 'Clasificacion_ABC_XYZ']

Total de ítems clasificados: 660

Ejemplo de clasificaciones:
         ITEM Clase_ABC Clase_XYZ Clasificacion_ABC_XYZ
547  ITEM 114         C         Z                    CZ
353  ITEM 244         B         Y                    BY
499  ITEM 570         C         Y                    CY
173  ITEM 341         A         Y                    AY
241  ITEM 427         A         Y                    AY

Distribución de clasificaciones ABC-XYZ:
Clasificacion_ABC_XYZ
AX     77
AY    116
AZ    106
BX     54
BY     40
BZ     53
CX     87
CY     68
CZ     59
Name: count, dtype: int64

Resultados exportados a 'Clasificacion_ABC_XYZ.xlsx'


In [17]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats.mstats import winsorize
from scipy.stats import norm

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# 1. Descargar y cargar datos
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer todas las hojas necesarias
historico_df = pd.read_excel(excel_data, sheet_name='Historico')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')
lead_time_df = pd.read_excel(excel_data, sheet_name='LeadTime-Dias')

print("="*80)
print("🚀 INICIANDO ANÁLISIS DE INVENTARIOS - ABC-XYZ + STOCK DE SEGURIDAD")
print("="*80)

# ============================================================================
# PARTE 1: CLASIFICACIÓN ABC
# ============================================================================
print("\n📊 Calculando Clasificación ABC...")

# Limpiar y preparar datos para ABC
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Calcular ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar y convertir precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[\$,]', '', regex=True).astype(float)

# Unir datos y calcular valor total
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y calcular acumulado
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
def clasificar_abc(porcentaje):
    if porcentaje <= 60: return 'A'
    elif porcentaje <= 80: return 'B'
    else: return 'C'

abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(clasificar_abc)
print(f"   ✅ Items clasificados ABC: {len(abc_df)}")

# ============================================================================
# PARTE 2: CLASIFICACIÓN XYZ
# ============================================================================
print("\n📈 Calculando Clasificación XYZ...")

# Seleccionar últimos 18 meses de datos históricos para XYZ
xyz_data = historico_df.set_index('ITEM').iloc[:, -18:].copy()

# Función para limpiar datos XYZ
def clean_row_xyz(row):
    clean = row.copy().astype(float)
    positive_vals = clean[clean > 0]
    median_val = positive_vals.median() if not positive_vals.empty else 0
    clean[clean <= 0] = median_val

    if clean.nunique() > 1:
        try:
            clean[:] = winsorize(clean, limits=[0.05, 0.05])
        except:
            pass
    return clean

# Aplicar limpieza
xyz_clean = xyz_data.apply(clean_row_xyz, axis=1)

# Calcular coeficiente de variación
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Clasificar XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)

xyz_df = pd.DataFrame({
    'ITEM': xyz_clean.index.tolist(),
    'CV': cv.values
}).reset_index(drop=True)

xyz_df['Clase_XYZ'] = pd.cut(cv.values, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])
xyz_df['ITEM'] = xyz_df['ITEM'].astype(str).str.strip()

print(f"   ✅ Items clasificados XYZ: {len(xyz_df)}")

# ============================================================================
# PARTE 3: COMBINAR ABC Y XYZ
# ============================================================================
print("\n🔄 Combinando clasificaciones ABC-XYZ...")

abc_minimal = abc_df[['ITEM', 'Clase_ABC']].copy()
abc_minimal['ITEM'] = abc_minimal['ITEM'].astype(str).str.strip()

clasificacion_df = abc_minimal.merge(xyz_df[['ITEM', 'Clase_XYZ']], on='ITEM', how='inner')
clasificacion_df['Clasificacion_ABC_XYZ'] = clasificacion_df['Clase_ABC'].astype(str) + clasificacion_df['Clase_XYZ'].astype(str)

print(f"   ✅ Items con clasificación completa: {len(clasificacion_df)}")

# ============================================================================
# PARTE 4: CONFIGURAR NIVELES DE SERVICIO POR CLASIFICACIÓN
# ============================================================================
print("\n⚙️ Configurando niveles de servicio por clasificación...")

# Diccionario de niveles de servicio según clasificación ABC-XYZ
niveles_servicio = {
    'AX': 0.95, 'AY': 0.90, 'AZ': 0.85,
    'BX': 0.85, 'BY': 0.80, 'BZ': 0.75,
    'CX': 0.75, 'CY': 0.70, 'CZ': 0.65
}

# Agregar nivel de servicio y Z-score
clasificacion_df['Nivel_Servicio'] = clasificacion_df['Clasificacion_ABC_XYZ'].map(niveles_servicio)
clasificacion_df['Z_Score'] = clasificacion_df['Nivel_Servicio'].apply(lambda x: norm.ppf(x) if pd.notna(x) else 1.645)

print("   📋 Niveles de servicio configurados:")
for clase, nivel in niveles_servicio.items():
    count = len(clasificacion_df[clasificacion_df['Clasificacion_ABC_XYZ'] == clase])
    print(f"      {clase}: {nivel*100:.0f}% (Z={norm.ppf(nivel):.3f}) - {count} items")

# ============================================================================
# PARTE 5: CALCULAR STOCK DE SEGURIDAD
# ============================================================================
print("\n🛡️ Calculando Stock de Seguridad...")

# Limpiar datos para stock de seguridad
historico_df['ITEM'] = historico_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Parámetros
R_dias = 0.5
WINSOR_LOW = 0.05
WINSOR_HIGH = 0.95
MIN_MESES_PARA_CALCULO = 12

# Lista para resultados de stock de seguridad
stock_resultados = []

print(f"   🔄 Procesando {len(clasificacion_df)} items...")

for idx, class_row in clasificacion_df.iterrows():
    item = class_row['ITEM']
    Z = class_row['Z_Score']

    # Buscar el item en datos históricos
    item_data = historico_df[historico_df['ITEM'] == item]

    if len(item_data) == 0:
        print(f"      ⚠️ Item {item} no encontrado en histórico")
        stock_resultados.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': np.nan,
            'Promedio_Ventas_Mensual': np.nan,
            'Desviacion_Ventas': np.nan,
            'Lead_Time_Dias': np.nan
        })
        continue

    # Extraer y convertir ventas
    ventas = pd.to_numeric(item_data.iloc[0, 1:], errors='coerce')

    # --- Paso 1: Imputar valores negativos ---
    ventas_imp = ventas.copy()

    negativos = ventas_imp < 0
    for i in negativos[negativos].index:
        try:
            pos = list(ventas.index).index(i)
        except:
            continue

        inicio_ventana = max(0, pos - 12)
        ventana = ventas.iloc[inicio_ventana:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]

        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos_positivos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos_positivos.mean() if len(todos_positivos) > 0 else 0

        ventas_imp[i] = max(0, estimado)

    ventas_imp = ventas_imp.clip(lower=0)

    # --- Paso 2: Tomar los últimos 30 meses ---
    ultimos_30_meses = ventas_imp[-30:]

    if len(ultimos_30_meses.dropna()) < MIN_MESES_PARA_CALCULO:
        stock_resultados.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': np.nan,
            'Promedio_Ventas_Mensual': np.nan,
            'Desviacion_Ventas': np.nan,
            'Lead_Time_Dias': np.nan
        })
        continue

    # --- Winsorización ---
    low_val = ultimos_30_meses.quantile(WINSOR_LOW)
    high_val = ultimos_30_meses.quantile(WINSOR_HIGH)
    ultimos_30_winsor = ultimos_30_meses.clip(lower=low_val, upper=high_val)

    # Calcular promedio y desviación estándar
    promedio_ventas = ultimos_30_winsor.mean()
    desviacion = ultimos_30_winsor.std()

    if pd.isna(desviacion) or desviacion == 0:
        desviacion = 0.1 * promedio_ventas if promedio_ventas > 0 else 0.1

    # --- Lead Time ---
    if item in lead_time_df['ITEM'].values:
        lt_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]
        lt_row = pd.to_numeric(lt_row, errors='coerce').dropna()
        if len(lt_row) > 0:
            promedio_lead_time_dias = lt_row.mean()
        else:
            promedio_lead_time_dias = 5
    else:
        promedio_lead_time_dias = 5

    # Convertir L y R a meses
    L = promedio_lead_time_dias / 30
    R = R_dias / 30

    # --- Calcular Stock de Seguridad ---
    try:
        SS_unidades = Z * desviacion * np.sqrt(L + R)
    except:
        SS_unidades = 0.1

    # Convertir a días
    if promedio_ventas > 0:
        stock_seguridad_dias = (SS_unidades / promedio_ventas) * 30
        stock_seguridad_dias = round(stock_seguridad_dias, 2)
    else:
        stock_seguridad_dias = 0.0

    stock_resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': stock_seguridad_dias,
        'Promedio_Ventas_Mensual': round(promedio_ventas, 2),
        'Desviacion_Ventas': round(desviacion, 2),
        'Lead_Time_Dias': round(promedio_lead_time_dias, 1)
    })

print(f"   ✅ Stock de seguridad calculado para {len(stock_resultados)} items")

# ============================================================================
# PARTE 6: CONSOLIDAR RESULTADOS FINALES
# ============================================================================
print("\n📋 Consolidando resultados finales...")

# Crear DataFrame de stock de seguridad
stock_df = pd.DataFrame(stock_resultados)

# Unir todos los resultados
resultado_final = clasificacion_df.merge(stock_df, on='ITEM', how='left')

# Agregar información adicional útil
resultado_final = resultado_final.merge(
    abc_df[['ITEM', 'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares']],
    on='ITEM', how='left'
)

# Reordenar columnas para mejor presentación
columnas_ordenadas = [
    'ITEM', 'Clasificacion_ABC_XYZ', 'Clase_ABC', 'Clase_XYZ',
    'Nivel_Servicio', 'Stock_Seguridad_Dias',
    'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares',
    'Promedio_Ventas_Mensual', 'Desviacion_Ventas', 'Lead_Time_Dias'
]

resultado_final = resultado_final[columnas_ordenadas]

# ============================================================================
# PARTE 7: EXPORTAR Y MOSTRAR RESULTADOS
# ============================================================================
print("\n💾 Exportando resultados...")

# Exportar a Excel
resultado_final.to_excel("Analisis_Completo_ABC_XYZ_Stock_Seguridad.xlsx", index=False)

print("="*80)
print("📊 RESUMEN DE RESULTADOS")
print("="*80)

print(f"\n🎯 Total de ítems analizados: {len(resultado_final)}")
print(f"📈 Items con datos completos: {len(resultado_final.dropna())}")

print("\n📊 Distribución por Clasificación ABC-XYZ:")
distribucion = resultado_final['Clasificacion_ABC_XYZ'].value_counts().sort_index()
for clase, cantidad in distribucion.items():
    nivel = niveles_servicio.get(clase, 0)
    print(f"   {clase}: {cantidad:3d} items (Nivel servicio: {nivel*100:.0f}%)")

print(f"\n💰 Stock de Seguridad Promedio por Clasificación (días):")
stock_promedio = resultado_final.groupby('Clasificacion_ABC_XYZ')['Stock_Seguridad_Dias'].agg(['mean', 'count']).round(1)
for clase, stats in stock_promedio.iterrows():
    if not pd.isna(stats['mean']):
        print(f"   {clase}: {stats['mean']:6.1f} días promedio ({int(stats['count'])} items)")

print(f"\n📋 Ejemplo de resultados:")
muestra = resultado_final[['ITEM', 'Clasificacion_ABC_XYZ', 'Nivel_Servicio', 'Stock_Seguridad_Dias', 'Valor_Total_Dolares']].head(10)
print(muestra.to_string(index=False))

print(f"\n✅ Resultados exportados a: 'Analisis_Completo_ABC_XYZ_Stock_Seguridad.xlsx'")
print("="*80)

🚀 INICIANDO ANÁLISIS DE INVENTARIOS - ABC-XYZ + STOCK DE SEGURIDAD

📊 Calculando Clasificación ABC...
   ✅ Items clasificados ABC: 660

📈 Calculando Clasificación XYZ...
   ✅ Items clasificados XYZ: 660

🔄 Combinando clasificaciones ABC-XYZ...
   ✅ Items con clasificación completa: 660

⚙️ Configurando niveles de servicio por clasificación...
   📋 Niveles de servicio configurados:
      AX: 95% (Z=1.645) - 77 items
      AY: 90% (Z=1.282) - 116 items
      AZ: 85% (Z=1.036) - 106 items
      BX: 85% (Z=1.036) - 54 items
      BY: 80% (Z=0.842) - 40 items
      BZ: 75% (Z=0.674) - 53 items
      CX: 75% (Z=0.674) - 87 items
      CY: 70% (Z=0.524) - 68 items
      CZ: 65% (Z=0.385) - 59 items

🛡️ Calculando Stock de Seguridad...
   🔄 Procesando 660 items...


/tmp/ipython-input-190412712.py:223: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ultimos_30_winsor = ultimos_30_meses.clip(lower=low_val, upper=high_val)
/tmp/ipython-input-190412712.py:223: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ultimos_30_winsor = ultimos_30_meses.clip(lower=low_val, 

   ✅ Stock de seguridad calculado para 660 items

📋 Consolidando resultados finales...

💾 Exportando resultados...
📊 RESUMEN DE RESULTADOS

🎯 Total de ítems analizados: 660
📈 Items con datos completos: 660

📊 Distribución por Clasificación ABC-XYZ:
   AX:  77 items (Nivel servicio: 95%)
   AY: 116 items (Nivel servicio: 90%)
   AZ: 106 items (Nivel servicio: 85%)
   BX:  54 items (Nivel servicio: 85%)
   BY:  40 items (Nivel servicio: 80%)
   BZ:  53 items (Nivel servicio: 75%)
   CX:  87 items (Nivel servicio: 75%)
   CY:  68 items (Nivel servicio: 70%)
   CZ:  59 items (Nivel servicio: 65%)

💰 Stock de Seguridad Promedio por Clasificación (días):
   AX:    3.3 días promedio (77 items)
   AY:    2.8 días promedio (116 items)
   AZ:    2.4 días promedio (106 items)
   BX:    2.3 días promedio (54 items)
   BY:    1.9 días promedio (40 items)
   BZ:    1.7 días promedio (53 items)
   CX:    1.4 días promedio (87 items)
   CY:    1.2 días promedio (68 items)
   CZ:    0.9 días promedio (